In [145]:
import torch
import glob
import cv2
import os
import numpy as np
import random
import time
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import ReduceLROnPlateau
from xml.dom.minidom import parse
from torchvision.models.detection.faster_rcnn import fasterrcnn_resnet50_fpn

In [146]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [147]:
idx2label = ["__background__", "person", "bicycle", "car", "motorbike", "bus"]
label2idx = {label:idx for idx, label in enumerate(idx2label)}

In [148]:
def get_image_data(data_dir = "./data/G3/"):
    image_data = dict()
    img_name_list = glob.glob(os.path.join(data_dir, "*.png"))
    for image_file in img_name_list:
        image_file = os.path.basename(image_file).split(".png")[0]
        image_filename = os.path.join(data_dir, image_file + ".png")
        image_data[image_file] = torch.Tensor(cv2.imread(image_filename)).permute(2,0,1)
    return image_data

In [149]:
def vis_data(img_name, image, target, data_dir="./data/pred/", isPred = True, show=True):
    image = np.array(image).astype(np.uint8)
    boxes = target['boxes'].cpu().numpy()
    labels = target['labels'].cpu().numpy()
    
    if isPred:
        scores = target["scores"].cpu().numpy()
        for i, score in enumerate(scores):
            if(score > 0.8):
                cv2.putText(image, idx2label[labels[i]], (int(boxes[i][0]), int(boxes[i][1])), cv2.FONT_HERSHEY_COMPLEX, 1.0, (0,0,255), 1)
                cv2.rectangle(image, (boxes[i][0], boxes[i][1]), (boxes[i][2], boxes[i][3]), tuple(random.randint(0,255) for _ in range(3)), 2)
    else:
        for i in range(len(boxes)):
            cv2.putText(image, idx2label[labels[i]], (int(boxes[i][0]), int(boxes[i][1])), cv2.FONT_HERSHEY_COMPLEX, 1.0, (0,0,255), 1)
            cv2.rectangle(image, (boxes[i][0], boxes[i][1]), (boxes[i][2], boxes[i][3]), tuple(random.randint(0,255) for _ in range(3)), 2)
    
    cv2.imwrite(os.path.join(data_dir, img_name + ".png"), image)
    
    if show:
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        plt.imshow(image)

In [150]:
def writelog(img_name, boxes, labels, scores, data_dir = "./data/pt/", threshold = 0.8):
    data_len = len(scores)
    with open(os.path.join(data_dir, img_name + ".txt"), "w") as f:
        for i in range(data_len):
            if scores[i] > threshold :
                f.write("{} {} {} {} {}\n".format(idx2label[labels[i]], boxes[i][0], boxes[i][1], boxes[i][2], boxes[i][3]))
    f.close()

In [151]:
def compute_IOU(rec1,rec2):
    #rec: rectangle, [xmin ymin xmax ymax]
    #return IoU of rec1 and rec2
    width=max(0,min(rec1[2],rec2[2])-max(rec1[0],rec2[0]))
    hight=max(0,min(rec1[3],rec2[3])-max(rec1[1],rec2[1]))
    inter=width*hight
    union=(rec1[3]-rec1[1])*(rec1[2]-rec1[0])+(rec2[3]-rec2[1])*(rec2[2]-rec2[0])-inter
    return inter/union

In [152]:
def evaluate_model_nobatch(image_data):
#     model = torch.load("best_model.pt", map_location=torch.device('cpu'))
    model.eval()
    
    for img_name in image_data:

        with torch.no_grad():
            if use_cuda:
                pred_data = model([image_data[img_name].cuda()])
            else:
                pred_data = model([image_data[img_name].cpu()])
        
        writelog(img_name = img_name, boxes = pred_data[0]["boxes"].cpu(), labels = pred_data[0]["labels"].cpu(), scores = pred_data[0]["scores"].cpu())
        vis_data(img_name = img_name, image = image_data[img_name].permute(1,2,0), target = pred_data[0], data_dir="./data/pred/", isPred = True, show=False)
        


In [153]:
image_data = get_image_data()

In [154]:
use_cuda = True

In [155]:
if use_cuda:
    model = torch.load("best_model_0.0211.pt", map_location=torch.device('cuda'))
else:
    model = torch.load("best_model_0.0211.pt", map_location=torch.device('cpu'))

In [156]:
evaluate_model_nobatch(image_data)

In [18]:
torch.cuda.empty_cache()